## PER + PBR 합성전략
- PBR < 1 : 자산 대비 저평가
- PER < 10 : 수익 대비 저평가

### 맞는 데이터 가져오고 merge 하기

In [1]:
from pykrx import stock

In [2]:
year = '2023'
df_first = stock.get_market_cap_by_ticker(f'{year}0102')
df_first = df_first[['종가', '시가총액']].rename(columns={'종가':'시가'})
df_first

,시가,시가총액
티커,,
005930,55500,331322931525000
373220,446000,104364000000000
207940,827000,58860898000000
000660,75700,55109779030500
051910,604000,42637775172000
...,...,...
215050,912,1934959392
288490,49,1869203000
267810,1010,1804870000


In [3]:
df_first.describe()

,시가,시가총액
count,2690.000000,2.690000e+03
mean,19262.999257,7.713959e+11
std,51201.159850,7.256943e+12
min,49.000000,1.178126e+09
25%,2636.250000,5.396002e+10
50%,6235.000000,1.092750e+11
75%,14925.000000,2.592859e+11
max,827000.000000,3.313229e+14


In [4]:
df_first = df_first.sort_values('시가총액') # 범위 나누기 정렬
df_first

,시가,시가총액
티커,,
322190,132,1178126004
267060,181,1654666524
267810,1010,1804870000
288490,49,1869203000
215050,912,1934959392
...,...,...
051910,604000,42637775172000
000660,75700,55109779030500
207940,827000,58860898000000


In [5]:
import pandas as pd
df_first['symbol_group'] = pd.cut(df_first.reset_index().index, bins = 3, labels=['소형주', '중형주', '대형주'])

In [6]:
df_first # symbol_group

,시가,시가총액,symbol_group
티커,,,
322190,132,1178126004,소형주
267060,181,1654666524,소형주
267810,1010,1804870000,소형주
288490,49,1869203000,소형주
215050,912,1934959392,소형주
...,...,...,...
051910,604000,42637775172000,대형주
000660,75700,55109779030500,대형주
207940,827000,58860898000000,대형주


### read PER + PBR from pykrx

In [7]:
df_second = stock.get_market_fundamental_by_ticker(f'{year}0102')
df_second.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
095570,8075,3.35,0.71,1707,4.72,270
006840,45961,0.00,0.35,0,1.23,200
027410,16393,6.02,0.25,684,2.67,110
282330,46849,23.63,4.31,8547,1.49,3000
138930,28745,2.69,0.22,2341,8.89,560


In [8]:
df_second = df_second[['BPS','PER']]
df_second

,BPS,PER
티커,,
095570,8075,3.35
006840,45961,0.00
027410,16393,6.02
282330,46849,23.63
138930,28745,2.69
...,...,...
005010,14549,5.67
000540,10599,3.88
000547,0,0.00


### 해당 년도 마지막 거래 종가

In [9]:
df_third = stock.get_market_ohlcv_by_ticker(f'{year}1228')
df_third = df_third[['종가']]

In [10]:
df_third

,종가
티커,
095570,4980
006840,17100
027410,3810
282330,131300
138930,7140
...,...
079980,4120
005010,4985
000540,3070


#### merge

In [11]:
df_concat = pd.concat([df_first, df_second, df_third], axis=1)
df_concat

,시가,시가총액,symbol_group,BPS,PER,종가
티커,,,,,,
322190,132.0,1.178126e+09,소형주,NaN,NaN,NaN
267060,181.0,1.654667e+09,소형주,NaN,NaN,NaN
267810,1010.0,1.804870e+09,소형주,NaN,NaN,NaN
288490,49.0,1.869203e+09,소형주,NaN,NaN,NaN
215050,912.0,1.934959e+09,소형주,NaN,NaN,NaN
...,...,...,...,...,...,...
45014K,NaN,NaN,NaN,NaN,NaN,5810.0
452260,NaN,NaN,NaN,NaN,NaN,1358.0
45226K,NaN,NaN,NaN,NaN,NaN,4365.0


In [12]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2708 entries, 322190 to 453340
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   시가            2690 non-null   float64 
 1   시가총액          2690 non-null   float64 
 2   symbol_group  2690 non-null   category
 3   BPS           914 non-null    float64 
 4   PER           914 non-null    float64 
 5   종가            953 non-null    float64 
dtypes: category(1), float64(5)
memory usage: 129.7+ KB


In [13]:
df_concat_dropna = df_concat.dropna()
df_concat_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 002787 to 005930
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   시가            905 non-null    float64 
 1   시가총액          905 non-null    float64 
 2   symbol_group  905 non-null    category
 3   BPS           905 non-null    float64 
 4   PER           905 non-null    float64 
 5   종가            905 non-null    float64 
dtypes: category(1), float64(5)
memory usage: 43.4+ KB


In [14]:
df_concat_dropna['수익률'] = df_concat_dropna['종가'] / df_concat_dropna['시가']
df_concat_dropna.describe()

/tmp/ipykernel_45151/1873410933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat_dropna['수익률'] = df_concat_dropna['종가'] / df_concat_dropna['시가']


,시가,시가총액,BPS,PER,종가,수익률
count,905.000000,9.050000e+02,9.050000e+02,905.000000,905.000000,905.000000
mean,38305.522652,1.918690e+12,4.875271e+04,19.922796,38432.748066,1.128948
std,81887.812583,1.241832e+13,1.874106e+05,222.605209,77110.309587,0.729142
min,199.000000,3.773542e+09,0.000000e+00,0.000000,165.000000,0.065122
25%,3745.000000,8.191163e+10,2.273000e+03,0.000000,3910.000000,0.864247
50%,11000.000000,2.174614e+11,1.109300e+04,4.940000,11550.000000,1.000000
75%,34550.000000,7.277092e+11,3.826200e+04,12.070000,37000.000000,1.192810
max,827000.000000,3.313229e+14,4.626787e+06,6650.000000,760000.000000,11.974359


In [15]:
conditions = (df_concat_dropna['PER'] >= 2.5) & (df_concat_dropna['PER'] <= 10)
# df_concat_dropna[conditions]
df_concat_dropna[conditions].sort_values('PBR').head(5)

KeyError: 'PBR'